In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# phishing_legitimate = pd.read_csv("/content/sample_data/phishing.csv")
phishing_legitimate = pd.read_csv(r"C:\Users\Rasagna\Desktop\final 4yr\phishing.csv")

In [3]:
df=phishing_legitimate
df.head()

,Index,UsingIP,LongURL,ShortURL,Symbol@,Redirecting//,PrefixSuffix-,SubDomains,HTTPS,DomainRegLen,...,UsingPopupWindow,IframeRedirection,AgeofDomain,DNSRecording,WebsiteTraffic,PageRank,GoogleIndex,LinksPointingToPage,StatsReport,class
0,0,1,1,1,1,1,-1,0,1,-1,...,1,1,-1,-1,0,-1,1,1,1,-1
1,1,1,0,1,1,1,-1,-1,-1,-1,...,1,1,1,-1,1,-1,1,0,-1,-1
2,2,1,0,1,1,1,-1,-1,-1,1,...,1,1,-1,-1,1,-1,1,-1,1,-1
3,3,1,0,-1,1,1,-1,1,1,-1,...,-1,1,-1,-1,0,-1,1,1,1,1
4,4,-1,0,-1,1,-1,-1,1,1,-1,...,1,1,1,1,1,-1,1,-1,-1,1


In [4]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

In [5]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

X = df.drop(columns=["class", "Index"])
y = df["class"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report

param_grid = {
    "C": [0.01, 0.1, 1, 10, 100],
    "solver": ["liblinear", "lbfgs", "newton-cg"],
    "max_iter": [100, 200, 300]
}

log_reg = GridSearchCV(LogisticRegression(), param_grid, cv=5, scoring="accuracy", n_jobs=-1)
log_reg.fit(X_train, y_train)

print("Best Params:", log_reg.best_params_)
print("Accuracy:", accuracy_score(y_test, log_reg.predict(X_test)))
print(classification_report(y_test, log_reg.predict(X_test)))


Best Params: {'C': 0.1, 'max_iter': 100, 'solver': 'newton-cg'}
Accuracy: 0.9335142469470827
              precision    recall  f1-score   support

           0       0.94      0.91      0.92       976
           1       0.93      0.95      0.94      1235

    accuracy                           0.93      2211
   macro avg       0.93      0.93      0.93      2211
weighted avg       0.93      0.93      0.93      2211



In [ ]:
from sklearn.tree import DecisionTreeClassifier

param_grid = {
    "criterion": ["gini", "entropy"],
    "max_depth": [5, 10, 20, None],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 4]
}

dt = GridSearchCV(DecisionTreeClassifier(), param_grid, cv=5, scoring="accuracy", n_jobs=-1)
dt.fit(X_train, y_train)

print("Best Params:", dt.best_params_)
print("Accuracy:", accuracy_score(y_test, dt.predict(X_test)))
print(classification_report(y_test, dt.predict(X_test)))


Best Params: {'criterion': 'entropy', 'max_depth': 20, 'min_samples_leaf': 1, 'min_samples_split': 2}
Accuracy: 0.9525101763907734
              precision    recall  f1-score   support

           0       0.95      0.94      0.95       976
           1       0.95      0.96      0.96      1235

    accuracy                           0.95      2211
   macro avg       0.95      0.95      0.95      2211
weighted avg       0.95      0.95      0.95      2211



In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score, classification_report
import numpy as np


param_dist = {
    "n_estimators": [100, 200, 300, 500],
    "max_depth": [10, 20, None],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 4]
}

rf_random = RandomizedSearchCV(
    RandomForestClassifier(),
    param_distributions=param_dist,
    n_iter=10,
    cv=5,
    scoring="accuracy",
    n_jobs=-1,
    random_state=42
)

# Train the model
rf_random.fit(X_train, y_train)

print("Best Params:", rf_random.best_params_)
print("Accuracy:", accuracy_score(y_test, rf_random.predict(X_test)))
print(classification_report(y_test, rf_random.predict(X_test)))


Best Params: {'n_estimators': 200, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_depth': None}
Accuracy: 0.9827904573791585
              precision    recall  f1-score   support

          -1       0.98      0.98      0.98       976
           1       0.98      0.98      0.98      1235

    accuracy                           0.98      2211
   macro avg       0.98      0.98      0.98      2211
weighted avg       0.98      0.98      0.98      2211


In [ ]:
import xgboost as xgb
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score, classification_report


param_dist = {
    "n_estimators": [100, 300, 500],
    "learning_rate": [0.01, 0.1, 0.2],
    "max_depth": [3, 6, 9],
    "subsample": [0.7, 0.8, 0.9],
    "colsample_bytree": [0.7, 0.8, 0.9]
}


xgb_random = RandomizedSearchCV(
    xgb.XGBClassifier(use_label_encoder=False, eval_metric="logloss"),
    param_distributions=param_dist,
    n_iter=10,
    cv=5,
    scoring="accuracy",
    n_jobs=-1,
    random_state=42
)


xgb_random.fit(X_train, y_train)


print("Best Params:", xgb_random.best_params_)
print("Accuracy:", accuracy_score(y_test, xgb_random.predict(X_test)))
print(classification_report(y_test, xgb_random.predict(X_test)))


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [16:40:48] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Best Params: {'subsample': 0.9, 'n_estimators': 500, 'max_depth': 9, 'learning_rate': 0.1, 'colsample_bytree': 0.9}
Accuracy: 0.9633649932157394
              precision    recall  f1-score   support

           0       0.96      0.95      0.96       976
           1       0.96      0.97      0.97      1235

    accuracy                           0.96      2211
   macro avg       0.96      0.96      0.96      2211
weighted avg       0.96      0.96      0.96      2211



In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score

param_dist = {
    "n_estimators": [100, 200, 300],
    "learning_rate": [0.01, 0.1, 0.2],
    "max_depth": [3, 5, 7],
    "min_samples_split": [2, 5],
    "min_samples_leaf": [1, 2]
}

gb = RandomizedSearchCV(
    GradientBoostingClassifier(),
    param_distributions=param_dist,
    n_iter=10,
    cv=3,
    scoring="accuracy",
    n_jobs=-1,
    random_state=42
)

gb.fit(X_train, y_train)
print("Best Params:", gb.best_params_)
print("Accuracy:", accuracy_score(y_test, gb.predict(X_test)))


Best Params: {'n_estimators': 200, 'min_samples_split': 2, 'min_samples_leaf': 2, 'max_depth': 7, 'learning_rate': 0.1}
Accuracy: 0.9656264133876075


In [9]:
pip install --upgrade scikit-learn scikeras

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from tensorflow.keras.callbacks import EarlyStopping

batch_sizes = [16, 32, 64]
epochs_list = [30, 50, 60]


results = []


kf = KFold(n_splits=3, shuffle=True, random_state=42)

early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)


best_score = 0
best_params = {}

for batch_size in batch_sizes:
    for epochs in epochs_list:
        cv_scores = []


        for train_index, val_index in kf.split(X_train):
            X_cv_train, X_cv_val = X_train[train_index], X_train[val_index]
            y_cv_train, y_cv_val = y_train[train_index], y_train[val_index]


            model = Sequential([
                Dense(64, activation="relu", input_shape=(X_train.shape[1],)),
                Dense(32, activation="relu"),
                Dense(16, activation="relu"),
                Dense(1, activation="sigmoid")
            ])
            model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])


            model.fit(
                X_cv_train, y_cv_train,
                batch_size=batch_size,
                epochs=epochs,
                validation_data=(X_cv_val, y_cv_val),
                callbacks=[early_stopping],
                verbose=0
            )

            y_cv_pred = (model.predict(X_cv_val) > 0.5).astype(int)
            cv_score = accuracy_score(y_cv_val, y_cv_pred)
            cv_scores.append(cv_score)


        mean_cv_score = np.mean(cv_scores)
        print(f"Batch size: {batch_size}, Epochs: {epochs}, Mean CV Score: {mean_cv_score:.4f}")


        if mean_cv_score > best_score:
            best_score = mean_cv_score
            best_params = {'batch_size': batch_size, 'epochs': epochs}

        results.append({
            'batch_size': batch_size,
            'epochs': epochs,
            'mean_cv_score': mean_cv_score
        })

print("\nBest Parameters:", best_params)
print("Best CV Score:", best_score)

final_model = Sequential([
    Dense(64, activation="relu", input_shape=(X_train.shape[1],)),
    Dense(32, activation="relu"),
    Dense(16, activation="relu"),
    Dense(1, activation="sigmoid")
])
final_model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

final_model.fit(
    X_train, y_train,
    batch_size=best_params['batch_size'],
    epochs=best_params['epochs'],
    verbose=0
)


y_pred = (final_model.predict(X_test) > 0.5).astype(int)
test_accuracy = accuracy_score(y_test, y_pred)
print(f"Test Accuracy with best parameters: {test_accuracy:.4f}")

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


...

9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
Batch size: 64, Epochs: 60, Mean CV Score: 0.8625

Best Parameters: {'batch_size': 16, 'epochs': 60}
Best CV Score: 0.8750058667267421


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
Test Accuracy with best parameters: 0.8550


In [12]:

print("Best Parameters: {'batch_size': 16, 'epochs': 60}")
print("Best CV Score: 0.8750")
print("Test Accuracy with best parameters: 0.8550")
print()
print("Hyperparameter Search Results:")
print("Best batch size: 16")
print("Best epochs: 60")
print("Best cross-validation accuracy: 0.8750")
print("Final test accuracy: 0.8550")


Best Parameters: {'batch_size': 16, 'epochs': 60}
Best CV Score: 0.8750
Test Accuracy with best parameters: 0.8550

Hyperparameter Search Results:
Best batch size: 16
Best epochs: 60
Best cross-validation accuracy: 0.8750
Final test accuracy: 0.8550


In [13]:
import pickle
with open("rf_model.pkl", "wb") as file:
    pickle.dump(rf_random, file)

print("Model saved successfully!")


Model saved successfully!


In [14]:
import pickle


with open("rf_model.pkl", "rb") as file:
    model = pickle.load(file)

print("Loaded Model Type:", type(model))

sample_features = [[1, 1, 1, 1, 1, 1, -1, -1, -1, -1, 1, 1, -1, -1, -1, -1, 1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 1, -1, 1]]
prediction = model.predict(sample_features)

print("Predicted Output:", prediction)


Loaded Model Type: <class 'sklearn.model_selection._search.RandomizedSearchCV'>
Predicted Output: [-1]


c:\Users\Rasagna\Desktop\final 4th year\venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


In [15]:
import pickle


with open("model.pkl", "wb") as file:
    pickle.dump(model, file)

print("Model saved correctly!")


Model saved correctly!


In [16]:
import pickle

with open("rf_model.pkl", "rb") as file:
    data = pickle.load(file)

print(type(data))  


<class 'sklearn.model_selection._search.RandomizedSearchCV'>
